# STM Transit Delay Data Preparation

## Overview

This notebook merges data from different sources and prepares it for data analysis and preprocessing.

## Data description

### Real-time Trip Updates

`current_time` timestamp when the data was collected<br>
`trip_id` unique identifier of a trip<br>
`route_id` bus line<br>
`start_date` start date of the trip<br>
`stop_id` stop number<br>
`arrival_time` actual arrival time, in milliseconds<br>
`departure_time` actual departure time, in milliseconds<br>
`schedule_relationship` state of the trip, 0 means scheduled and 1 means skipped

### Scheduled STM Trips

`trip_id` unique identifier of a trip<br>
`arrival_time` scheduled arrival time, in milliseconds<br>
`departure_time` scheduled departure time, in milliseconds<br>
`stop_id` stop number<br>
`stop_sequence` sequence of the stop, for ordering

### STM Stops

`stop_id` unique identifier of a stop<br>
`stop_code` bus stop or metro station number<br>
`stop_name` bus stop or metro station name<br>
`stop_lat` stop latitude<br>
`stop_lon` stop longitude<br>
`stop_url` stop web page<br>
`location_type` stop type<br>
`parent_station` parent station (metro station with multiple exits)<br>
`wheelchair_boarding` indicates if the stop is accessible for people in wheelchair, 1 being true and 2 being false

### Real-time Vehicle Positions

`current_time` timestamp when the data was collected<br>
`vehicle_id` unique identifuer of a vehicle<br>
`trip_id` unique identifier of a trip<br>
`route_id` bus or metro line<br>
`start_date` start date of a trip<br>
`start_time` start time of a trip<br>
`latitude` vehicle current latitude<br>
`longitude` vehicle current longityde<br>
`bearing` direction that the vehicle is facing<br>
`speed` momentary speed measured by the vehicle, in meters per second<br>
`stop_sequence` sequence of the stop, for ordering<br>
`status` vehicle stop status in relation with a stop that it's currently approaching or is at<br>
`timestamp` timestamp when STM updated the data<br>
`occupancy_status` degree of passenger occupancy

### Weather Archive and Forecast

`time` date and hour or the weather<br>
`temperature` air temperature at 2 meters above ground, in Celsius<br>
`precipitation` total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters<br>
`windspeed` wind speed at 10 meters above ground, in km/h<br>
`weathercode` World Meteorological Organization (WMO) code

### Traffic Incidents

`category` category of the incident<br>
`start_time` start time of the incident in ISO8601 format<br>
`end_time` end time of the incident in ISO8601 format<br>
`length` length of the incident in meters<br>
`delay` delay in seconds caused by the incident (except road closures)<br>
`magnitude_of_delay` severity of the delay<br>
`last_report_time` date in ISO8601 format, when the last time the incident was reported<br>
`latitude` latitude of the incident<br>
`longitude` longitude of the incident

## Imports

In [ ]:
from datetime import datetime, timedelta, timezone
from haversine import haversine, Unit
import pandas as pd
import sys

In [ ]:
# Import custom code
sys.path.insert(0, '..')
from scripts.custom_functions import fetch_weather, LOCAL_TIMEZONE

In [ ]:
trips_df = pd.read_csv('../data/api/fetched_stm_trip_updates.csv', low_memory=False)

In [ ]:
# TODO: use all schedule files using glob
schedules_df = pd.read_csv('../data/download/stop_times_2025-04-23.txt')

In [ ]:
stops_df = pd.read_csv('../data/download/stops_2025-04-23.txt')

In [ ]:
positions_df =  pd.read_csv('../data/api/fetched_stm_vehicle_positions.csv', low_memory=False)

In [ ]:
weather_df = pd.read_csv('../data/api/fetched_historical_weather.csv')

In [ ]:
traffic_df = pd.read_csv('../data/api/fetched_traffic.csv')

## Clean Data

In [ ]:
# Convert route_id to integer
trips_df['route_id'] = trips_df['route_id'].str.extract(r'(\d+)')
trips_df['route_id'] = trips_df['route_id'].astype('int64')

In [ ]:
# Sort trips
trips_df = trips_df.sort_values(by=['current_time', 'trip_id', 'route_id', 'arrival_time'])

In [ ]:
# Get proportion of duplicates
subset = ['start_date', 'trip_id', 'route_id', 'stop_id']
duplicate_mask = trips_df.duplicated(subset=subset)
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
trips_df = trips_df.drop_duplicates(subset=subset, keep='last') # keep latest update

In [ ]:
# Convert realtime arrival and departure time to milliseconds
trips_df['arrival_time'] = trips_df['arrival_time'] * 1000
trips_df['departure_time'] = trips_df['departure_time'] * 1000

In [ ]:
# Get distribution of realtime arrival times
trips_df[['arrival_time', 'departure_time']].describe()

In [ ]:
# Get proportion of rows with zero arrival times
zero_mask = trips_df['arrival_time'] == 0
print(f'{zero_mask.mean():.2%}')

In [ ]:
# Get proportion of rows where the arrival and departure times are different
diff_date_mask = trips_df['arrival_time'] != trips_df['departure_time']
print(f'{diff_date_mask.mean():.2%}')

In [ ]:
# Display rows
trips_df[diff_date_mask].sample(10)

In [ ]:
# Replace zero arrival times by departure times, as they are usually the same
trips_df.loc[zero_mask, 'arrival_time'] = trips_df.loc[zero_mask, 'departure_time']

In [ ]:
# Get proportion of rows with zero arrival times again
zero_mask = trips_df['arrival_time'] == 0
print(f'{zero_mask.mean():.2%}')

In [ ]:
# Delete the rows with 0 arrival times
trips_df = trips_df[~zero_mask]
zero_mask = trips_df['arrival_time'] == 0
assert zero_mask.sum() == 0

In [ ]:
# Rename arrival time
trips_df = trips_df.rename(columns={'arrival_time': 'realtime_arrival_time'})

## Merge Data

### Realtime and Scheduled Trips

In [ ]:
# Sort values by stop sequence
schedules_df = schedules_df.sort_values(by=['trip_id', 'stop_sequence'])

In [ ]:
# Reset stop sequences (some stops might be missing)
schedules_df['stop_sequence'] = schedules_df.groupby('trip_id').cumcount() + 1

In [ ]:
# Add trip progress (vehicles further along the trip are more likely to be delayed)
total_stops = schedules_df.groupby('trip_id')['stop_id'].transform('count')
schedules_df['trip_progress'] = schedules_df['stop_sequence'] / total_stops

In [ ]:
# Merge realtime and scheduled trips (#TODO: concatenate new schedule before left joining)
stm_trips_df = pd.merge(left=trips_df, right=schedules_df, how='inner', on=['trip_id', 'stop_id'])

In [ ]:
# Convert start_date to datetime
stm_trips_df['start_date_dt'] = pd.to_datetime(stm_trips_df['start_date'], format='%Y%m%d')

In [ ]:
def parse_gtfs_time(row) -> pd.Timestamp:
	'''
	Converts GTFS time string (e.g., '25:30:00') to datetime
	based on the arrival time.
	'''
	hours, minutes, seconds = map(int, row['arrival_time'].split(':'))
	total_seconds = hours * 3600 + minutes * 60 + seconds

	parsed_time = row['start_date_dt'] + timedelta(seconds=total_seconds)
	return parsed_time

In [ ]:
# Convert planned arrival time to localized datetime
parsed_time = stm_trips_df.apply(parse_gtfs_time, axis=1)
sch_arrival_time = parsed_time.dt.tz_localize(LOCAL_TIMEZONE)

In [ ]:
# Convert planned time to timestamp in milliseconds since epoch
stm_trips_df['scheduled_arrival_time'] = sch_arrival_time.astype('int64') // 10**6

In [ ]:
# Convert realtime and scheduled arrival time to UTC datetime
stm_trips_df['realtime_arrival_time_dt'] = pd.to_datetime(stm_trips_df['realtime_arrival_time'], origin='unix', unit='ms', utc=True)
stm_trips_df['scheduled_arrival_time_dt'] = pd.to_datetime(stm_trips_df['scheduled_arrival_time'], origin='unix', unit='ms', utc=True)

In [ ]:
# Calculate delay in seconds (real - scheduled)
stm_trips_df['delay'] = (stm_trips_df['realtime_arrival_time_dt'] - stm_trips_df['scheduled_arrival_time_dt']).dt.total_seconds()

### Trips and Stops

In [ ]:
trips_stops_df = pd.merge(left=stm_trips_df, right=stops_df, how='inner', left_on='stop_id', right_on='stop_code')

In [ ]:
trips_stops_df.columns

In [ ]:
# Rename stop id
trips_stops_df = trips_stops_df.rename(columns={'stop_id_x': 'stop_id'})

In [ ]:
# Convert wheelchair_boarding to boolean
trips_stops_df['wheelchair_boarding'] = (trips_stops_df['wheelchair_boarding'] == 1).astype('int64')

### Vehicle Positions

In [ ]:
# Rename latitude and longitude
positions_df = positions_df.rename(columns={'latitude': 'vehicle_lat', 'longitude': 'vehicle_lon'})

In [ ]:
# Sort values
subset = ['trip_id', 'route_id', 'start_date', 'stop_sequence']
positions_df = positions_df.sort_values(by=subset)

In [ ]:
# Get duplicates
duplicate_mask = positions_df.duplicated(subset=subset)
positions_df[duplicate_mask]

In [ ]:
# Get proportion of duplicates
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
positions_df = positions_df.drop_duplicates(subset=subset)

In [ ]:
# Merge with other STM data
stm_df = pd.merge(left=trips_stops_df, right=positions_df, how='inner', on=['trip_id', 'route_id', 'start_date', 'stop_sequence'])

In [ ]:
# Calculate distance between the vehicle and the stop
stm_df['vehicle_distance'] = stm_df.apply(
  lambda row: haversine((row['vehicle_lat'], row['vehicle_lon']), (row['stop_lat'], row['stop_lon']), unit=Unit.METERS),
  axis=1)

In [ ]:
stm_df['vehicle_distance'].describe()

In [ ]:
stm_df.info()

### STM and Weather

In [ ]:
# Convert time string to datetime
time_dt = pd.to_datetime(weather_df['time'], utc=True)

In [ ]:
# Convert STM arrival timestamp to datetime 
stm_df['arrival_time_dt'] = pd.to_datetime(stm_df['realtime_arrival_time'], origin='unix', unit='ms', utc=True)

In [ ]:
# Calculate dates for weather forecast
last_day_weather = time_dt.max()
start_date = last_day_weather + timedelta(days=1)
end_date = stm_df['arrival_time_dt'].max()

In [ ]:
# Fetch forecast weather
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

forecast_list = fetch_weather(start_date=start_date_str, end_date=end_date_str, forecast=True)
forecast_df = pd.DataFrame(forecast_list)

In [ ]:
# Merge archive and forecast weather
weather_df = pd.concat([weather_df, forecast_df], ignore_index=True)

In [ ]:
# Round arrival time to the nearest hour
stm_df['rounded_arrival_dt'] = stm_df['arrival_time_dt'].dt.round('h')

In [ ]:
# Format time to match weather data
stm_df['time'] = stm_df['rounded_arrival_dt'].dt.strftime('%Y-%m-%dT%H:%M')

In [ ]:
# Merge STM with weather
stm_weather_df = pd.merge(left=stm_df, right=weather_df, how='inner', on='time')

### Traffic Data

In [ ]:
# Get proportion of duplicates
duplicate_mask = traffic_df.duplicated()
print(f'{duplicate_mask.mean():.2%}')

In [ ]:
# Remove duplicates
traffic_df = traffic_df.drop_duplicates(keep='last').reset_index()

In [ ]:
# Convert traffic start_time and end_time to datetime
traffic_df['start_time_dt'] = pd.to_datetime(traffic_df['start_time'], utc=True)
traffic_df['end_time_dt'] = pd.to_datetime(traffic_df['end_time'], utc=True)

In [ ]:
# Sort by date
traffic_df = traffic_df.sort_values(by='start_time_dt').reset_index()

In [ ]:
# Fill null end times with current time (assuming the incident is still ongoing)
traffic_df['end_time_dt'] = traffic_df['end_time_dt'].fillna(datetime.now(timezone.utc).replace(microsecond=0))
assert traffic_df['end_time_dt'].isna().sum() == 0

In [ ]:
# Build traffic cache (for every 30 min interval)
def build_traffic_cache(traffic_df:pd.DataFrame) -> dict:
	traffic_cache = {}
	traffic_df['half_hour'] = traffic_df['start_time_dt'].dt.floor('30min')

	for (hour, group) in traffic_df.groupby('half_hour'):
		traffic_cache[hour] = group.copy()

	return traffic_cache

Since there are many trip updates on the same day (even the same hour), there's a risk of repeating the filtering of active traffic incidents for each trip individually, which takes a lot of time for a large dataset. Traffic incidents are stable over minutes or hours. This is why the incidents are cached by 30 minute intervals.

In [ ]:
def calculate_nearby_incidents(trip_update:pd.Series, traffic_cache:dict, max_distance:int=500) -> pd.Series:
	trip_datetime = trip_update['arrival_time_dt']
	stop_coords = (trip_update['vehicle_lat'], trip_update['vehicle_lon'])

	trip_half_hour = trip_datetime.floor('30min')

	# Get cached incidents
	hour_incidents = traffic_cache.get(trip_half_hour)

	# Stop if there are no incidents for that hour
	if hour_incidents is None or hour_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Filter for active incidents at that trip hour
	active_incidents = hour_incidents[
		(hour_incidents['start_time_dt'] <= trip_datetime) &
		(hour_incidents['end_time_dt'] >= trip_datetime)
	].copy()

	if active_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Calculate distance     
	active_incidents['distance'] = active_incidents.apply(
		lambda row: haversine(stop_coords, (row['latitude'], row['longitude']), unit=Unit.METERS),
		axis=1
	)

	# Filter nearby
	nearby_incidents = active_incidents[active_incidents['distance'] <= max_distance]

	if nearby_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})
	else:
		nearest = nearby_incidents.loc[nearby_incidents['distance'].idxmin()]
		return pd.Series({
			'incident_nearby': 1,
			'nearest_incident_distance': nearest['distance'],
			'incident_category': nearest['category'],
			'incident_delay': nearest['delay'],
			'incident_delay_magnitude': nearest['magnitude_of_delay']
		})

In [ ]:
# Get traffic columns (get incidents within 500 meters)
traffic_cache = build_traffic_cache(traffic_df)
traffic_cols = stm_weather_df.apply(lambda row: calculate_nearby_incidents(row, traffic_cache), axis=1)

In [ ]:
# Merge the traffic
df = pd.concat([stm_weather_df, traffic_cols], axis=1)

## Export Data

In [ ]:
# Remove columns with constant values or with more than 50% missing values
df = df.loc[:, (df.nunique() > 1) & (df.isna().mean() < 0.5)]
df.columns

In [ ]:
# Keep relevant columns
df = df[[
  	'vehicle_id', 
    'vehicle_lat',
    'vehicle_lon',
    'vehicle_distance',
	'occupancy_status',
  	'route_id',
  	'stop_id',
  	'stop_lat',
  	'stop_lon',
	'stop_sequence',
  	'trip_progress',
  	'wheelchair_boarding',
  	'realtime_arrival_time',
    'scheduled_arrival_time',
    'delay',
  	'temperature',
  	'precipitation',
  	'windspeed', 
	'weathercode',
  	'incident_nearby', 
	#'incident_category',
	#'nearest_incident_distance',
	#'incident_delay',
	#'incident_delay_magnitude'
	
]]

In [ ]:
df.info()

In [ ]:
# Export data to CSV
df.to_csv('../data/stm_weather_traffic_merged.csv', index=False)

## End